In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [ ]:
df = pd.read_csv('/kaggle/input/fifa-20-complete-player-dataset/players_20.csv')
df.head()

In [ ]:
df.team_position.fillna('NA',inplace = True)

In [ ]:
df.team_position.unique()

In [ ]:
df[df.team_position.isin(['RW', 'LW', 'CAM','RCM' ,'CDM', 'LDM', 'RM' ,'LCM','LM' ,'RDM' , 'RAM','CM','LAM'])]

In [ ]:
conditions = [
(df.team_position.isin([ 'LCB','RCB','LB','RB','RWB', 'LWB','CB'])),
(df.team_position.isin(['RW', 'LW','CAM','RCM' ,'CDM', 'LDM', 'RM' ,'LCM','LM','RDM','RAM','CM','LAM'])),
(df.team_position.isin(['GK'])),
(df.team_position.isin([  'ST','CF','LS','RS', 'RF', 'LF'])),
(df.team_position.isin(['RES','SUB']))]
values = ['Defender' , 'Midfielder' , 'Goalkeeper' , 'Attacker' , 'Sub/Reserves']
df['Position'] = np.select(conditions,values)
df.Position

In [ ]:
nullCheck =[df.club.isna().any(),
df.age.isna().any(),
df.overall.isna().any(),
df.wage_eur.isna().any(),df.team_position.isna().any()]
nullCheck

In [ ]:
clubs = ['Manchester City','Manchester United' ,'Liverpool','Arsenal' ,
         'Leeds United' ,'Leicester City' ,'Tottenham Hotspur' , 'Chelsea' ,'Everton']

In [ ]:
import plotly.graph_objects as go
import plotly.offline as offline
from plotly.offline import init_notebook_mode, iplot

clubs = ['Manchester City','Manchester United' ,'Liverpool','Arsenal' ,
         'Leeds United' ,'Leicester City' ,'Tottenham Hotspur' , 'Chelsea' ,'Everton']

fig = go.Figure()

# fig.add_trace(go.Scatter(x=df_grp[df_grp.club == 'Arsenal'].age, 
#                               y=df_grp[df_grp.club == 'Arsenal'].wage_eur,mode="markers",
#                          marker = dict(size=df_grp[df_grp.club == 'Arsenal'].overall
#                                       )
#                         ))

layout_club = []
layout_club.append(dict(label = 'All' , method = 'update' , args = [{"visible":[True] * len(clubs)}]))
for n,club in enumerate(clubs):
    visible = [False] * len(clubs)
    visible[n] = True
    temp = dict(label = club , method = 'update' , args = [{"visible":visible}])
    layout_club.append(temp)

#REVISIT LATER   
# layout_position = []
# layout_position.append(dict(label = 'All' , method = 'update' ,
#                             args = [{"visible":[True] * len(df.Position.unique())}]))
# for n,pos in enumerate(df.Position.unique()):
#     visible = [False] * len(df.Position.unique())
#     visible[n] = True
#     temp = dict(label = pos , method = 'update' , args = [{"visible":visible}])
#     layout_position.append(temp)
 
for club in clubs:
    fig.add_trace(go.Scatter(x=df[df.club == club].age,
                             y=df[df.club == club].wage_eur,mode="markers",
                             visible = True,
                             text = df[df.club == club].short_name,
                             customdata = np.stack((df[df.club == club].player_positions,
                                                    df[df.club == club].team_position,df[df.club == club].club
                                                    ,df[df.club == club].nationality ) , axis =1),
                             hovertemplate = 
                             "<b>%{text}</b><br><br>" +
                             "Age: %{x}<br>" +
                             "Wage: %{y}<br>" +
                             "Player positions : %{customdata[0]}<br>" + 
                             "Team Position : %{customdata[1]}<br>" + 
                             "Club : %{customdata[2]}<br>" +
                             "Nationality : %{customdata[3]}<br>" + 
                             "Overall: %{marker.size:,}" +
                             "<extra></extra>",
                             marker = dict(size=df[df.club == club].overall,
                             color=np.random.randn(100),
                             colorscale='icefire')
                        ))
    fig.update_traces(
    mode='markers',
    marker={'sizemode':'diameter',
            'sizeref':2})
fig.update_layout(template = 'plotly_white' , 
                  hovermode = 'closest',updatemenus = [dict(buttons = layout_club )],
                  xaxis=dict(title = 'Age', range=[16,40]),
                  yaxis=dict(title = 'Wage value in euros',range=[-100000,500000]),
                  title = "Wage x Age",showlegend=False
                  )
fig.show()

**Below, created new column order so that for every age we get all the bubbles from each team. Could have simply used the age column as well, but felt this is more discrete**

In [ ]:
dfa = df[df.club.isin(['Manchester City','Manchester United' , 
                       'Liverpool' , 'Tottenham Hotspur' ,
                       'Arsenal' , 'Chelsea'])][['short_name','club','age','overall','wage_eur','value_eur']]
def newcol(row):
    if row['club'] == 'Manchester City':
        return row['age']/10
    elif row['club']  == 'Manchester United':
        return row['age']/10
    elif row['club'] == 'Liverpool':
        return row['age']/10
    elif row['club'] == 'Tottenham Hotspur':
        return row['age']/10
    elif row['club'] == 'Arsenal':
        return row['age']/10
    elif row['club'] == 'Chelsea':
        return row['age']/10
dfa['order'] = dfa.apply(newcol,axis=1)

#dfa['order'] = np.arange(0,len(dfa),1)
dfa = dfa.sort_values(['order'])
dfa

In [ ]:
import plotly.express as px
fig = px.scatter(dfa, x="age", y="wage_eur",color="club",opacity=[1], 
                 range_x = (17,37),range_y=(0,400000),
                 size='value_eur', template = 'plotly_dark' , hover_data=['overall' , 'short_name'])
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(dfa, x="age", y="wage_eur",color="club",opacity=[1], 
                 animation_frame="order", animation_group = "short_name",
                 range_x = (17,37),range_y=(0,400000),
                 size='value_eur', template = 'plotly_dark' , hover_data=['overall' , 'short_name'])
fig.show()